# Wrapping a basic library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider a basic example of a *C++* library.

First, we need:

* to detect if the operating system (OS) is a Windows OS or a Unix OS.

In [1]:
import platform
is_windows = any(platform.win32_ver())

* to detect the version of *Python* installed and save it in the `PYTHON_VERSION` environment variable.

In [2]:
import os
import sys
os.environ['PYTHON_VERSION'] = str(sys.version_info.major) + '.' + str(sys.version_info.minor)

* to import **AutoWIG**.

In [3]:
import autowig

Then, we need to install and compile the *C++* library.
To do so, we use available **Conda** recipes.

In [4]:
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\..\bin\conda\libbasic -c statiskit -c conda-forge
else:
    !conda build --python=$PYTHON_VERSION ../../bin/conda/libbasic -c statiskit -c conda-forge
!conda install -y libbasic --use-local -c statiskit -c conda-forge --force

BUILD START: libbasic-1.0.0-0
    (actual version deferred until further download or env creation)
updating index in: /home/main/miniconda/conda-bld/linux-64
updating index in: /home/main/miniconda/conda-bld/noarch

The following NEW packages will be INSTALLED:

    ca-certificates: 2017.7.27.1-0      conda-forge
    certifi:         2017.7.27.1-py36_0 conda-forge
    libtoolchain:    1.0.0-py36_0       statiskit  
    ncurses:         5.9-10             conda-forge
    openssl:         1.0.2l-0           conda-forge
    path.py:         10.3.1-py36_0      conda-forge
    pip:             9.0.1-py36_0       conda-forge
    python:          3.6.2-0            conda-forge
    python-scons:    3.0.0-py36_0       statiskit  
    pyyaml:          3.12-py36_1        conda-forge
    readline:        6.2-0              conda-forge
    scons-tools:     1.0.0-py36_0       statiskit  
    setuptools:      36.3.0-py36_0      conda-forge
    six:             1.11.0-py36_1      conda-forge
    sqlit

Once these preliminaries are done, we can proceed to the actual generation of wrappers for the basic *C++* library.
For this, we create an empty Abstract Semantic Graph (ASG).

In [5]:
asg = autowig.AbstractSemanticGraph()

We then parse the headers of the basic *C++* library with relevant compilation flags.

In [6]:
%%time

try:
    from path import path as Path
except:
    from path import Path
prefix = Path(sys.prefix).abspath()
if is_windows:
    headers = [prefix/'Library'/'include'/'basic'/'overload.h',
               prefix/'Library'/'include'/'basic'/'binomial.h']
else:
    headers = [prefix/'include'/'basic'/'overload.h',
               prefix/'include'/'basic'/'binomial.h']
    
flags = ['-x', 'c++', '-std=c++11']
if is_windows:
    flags.append('-I' + str((prefix/'Library'/'include').abspath()))
else:
    flags.append('-I' + str((prefix/'include').abspath()))
    
asg = autowig.parser(asg, headers,
                          flags,
                          silent = True)

CPU times: user 210 ms, sys: 20 ms, total: 230 ms
Wall time: 860 ms


Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is suitable.

In [7]:
%%time
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

CPU times: user 80 ms, sys: 10 ms, total: 90 ms
Wall time: 87 ms


In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [8]:
%%time
autowig.generator.plugin = 'boost_python_internal'  
wrappers = autowig.generator(asg,
                             module = Path('.')/'..'/'..'/'src'/'py'/'_basic.cpp',
                             decorator = Path('.')/'..'/'..'/'src'/'py'/'basic'/'_basic.py',
                             prefix = 'wrapper_')

CPU times: user 80 ms, sys: 0 ns, total: 80 ms
Wall time: 76.8 ms


The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [9]:
%%time
wrappers.write()

CPU times: user 420 ms, sys: 580 ms, total: 1 s
Wall time: 3.21 s


Here is the list of the generated wrappers (untracked files).

In [10]:
!git status

On branch master
Your branch is up-to-date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	../../src/py/_basic.cpp
	../../src/py/_basic.h
	../../src/py/basic/_basic.py
	../../src/py/wrapper_0f744e8d056f5d469a887c7c78eaf8fe.cpp
	../../src/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp
	../../src/py/wrapper_92d464d1243554009adc48a065a5be3e.cpp
	../../src/py/wrapper_f926cb231a7f5da09f313cd361ff94c7.cpp

nothing added to commit but untracked files present (use "git add" to track)


And here, we present the wrappers for the `BinomialDistribution` class.

In [11]:
if is_windows:
    !pygmentize ..\..\src\py\wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp
else:
    !pygmentize ../../src/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp

#include "_basic.h"



namespace autowig
{

}

#if defined(_MSC_VER)
    #if (_MSC_VER == 1900)
namespace boost
{
    template <> class ::BinomialDistribution const volatile * get_pointer<class ::BinomialDistribution const volatile >(class ::BinomialDistribution const volatile *c) { return c; }
}
    #endif
#endif



void wrapper_4046a8421fe9587c9dfbc97778162c7d()
{

    double  (::BinomialDistribution::*method_pointer_3a3ff64f25e358a6a10b1cd3b3425b82)(unsigned int const) const = &::BinomialDistribution::pmf;
    double  (::BinomialDistribution::*method_pointer_3c97a500c9575c259d5cbdd76120ff4f)() const = &::BinomialDistribution::get_pi;
    void  (::BinomialDistribution::*method_pointer_d364a0529e33516f8ecbb7dcedd60aa0)(double const) = &::BinomialDistribution::set_pi;
    boost::python::class_< class ::BinomialDistribution, autowig::Held< class ::BinomialDistribution >::Type > class_4046a8421fe9587c9dfbc97778162c7d("BinomialDistribution", "", boost::python::no_init);
    class_4046a842

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [12]:
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\..\bin\conda\python-basic -c statiskit -c conda-forge
else:
    !conda build --python=$PYTHON_VERSION ../../bin/conda/python-basic -c statiskit -c conda-forge
!conda install -y python-basic --use-local -c statiskit --force

BUILD START: python-basic-1.0.0-py27_0
    (actual version deferred until further download or env creation)

The following NEW packages will be INSTALLED:

    alabaster:                0.7.10-py27_1      conda-forge
    asn1crypto:               0.22.0-py27_0      conda-forge
    babel:                    2.4.0-py27_0       conda-forge
    bzip2:                    1.0.6-1            conda-forge
    ca-certificates:          2017.7.27.1-0      conda-forge
    certifi:                  2017.7.27.1-py27_0 conda-forge
    cffi:                     1.10.0-py27_0      conda-forge
    chardet:                  3.0.4-py27_0       conda-forge
    coverage:                 4.4.1-py27_0       conda-forge
    cryptography:             2.0.3-py27_0       conda-forge
    docutils:                 0.14-py27_0        conda-forge
    enum34:                   1.1.6-py27_1       conda-forge
    freetype:                 2.7-1              conda-forge
    icu:                      58.2-h211956c_0      

g++ -o /home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/lib/libbasic.so -shared build/cpp/binomial.os build/cpp/overload.os -L/home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/lib
Install file: "src/py/basic/_basic.py" as "/home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/lib/python2.7/site-packages/basic/_basic.py"
Install file: "src/py/basic/__init__.py" as "/home/main/minicon

In file included from /home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/converter/arg_to_python.hpp:10:0,
                 from /home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/call.hpp:15,
                 from /home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/object_core.hpp:14,
                 from /home/main/min

In file included from /home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python.hpp:41:0,
                 from build/py/_basic.h:4:
/home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/make_constructor.hpp: In member function 'void boost::python::detail::install_holder<T>::dispatch(U*, mpl_::true_) const':
/home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla

In file included from /home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/object/function_handle.hpp:10:0,
                 from /home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/converter/arg_to_python.hpp:19,
                 from /home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/call.hpp:15,
                 from /ho

...

In file included from /home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python.hpp:41:0,
                 from build/py/_basic.h:4,
                 from build/py/wrapper_0f744e8d056f5d469a887c7c78eaf8fe.cpp:1:
/home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/make_constructor.hpp: In member function 'void boost::python::detail::install_holder<T>::dispatch(U*, mpl_::true_) const':
/home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac

g++ -o build/py/_basic.os -c -x c++ -std=c++11 -fvisibility=hidden -fmax-errors=0 -Wl,--no-undefined -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -I/home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include -I/home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/python2.7 build/py/_basic.cpp
In file included from /home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include

In file included from /home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/converter/arg_to_python.hpp:10:0,
                 from /home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/call.hpp:15,
                 from /home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/object_core.hpp:14,
                 from /home/main/min

In file included from /home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python.hpp:41:0,
                 from build/py/_basic.h:4,
                 from build/py/_basic.cpp:1:
/home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/include/boost/python/make_constructor.hpp: In member function 'void boost::python::detail::install_holder<T>::dispatch(U*, mpl_::true_) const':
/home/main/miniconda/conda-bld/python-basic_1507491197207/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehol

# Automatic uploading is disabled
# If you want to upload package(s) to anaconda.org later, type:

anaconda upload /home/main/miniconda/conda-bld/linux-64/python-basic-1.0.0-py27_0.tar.bz2

# To have conda build upload to anaconda.org automatically, use
# $ conda config --set anaconda_upload yes

anaconda_upload is not set.  Not uploading wheels: []
Fetching package metadata ...............
Solving package specifications: .

Package plan for installation in environment /home/main/miniconda:

The following NEW packages will be INSTALLED:

    python-basic: 1.0.0-py27_0 local



Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [13]:
import basic
binomial = basic.BinomialDistribution(1, .5)
binomial

In [14]:
binomial.pmf(0)

0.5

In [15]:
binomial.pmf(1)

0.5

In [16]:
binomial.n = 0
binomial

In [17]:
binomial.pmf(0)

1.0

In [18]:
try:
    binomial.set_pi(1.1)
except basic.ProbabilityError as error:
    print(error)
else:
    raise Exception('A `basic.ProbabilityError` should have been raised')

a probability must be in the interval [0,1]


Here is a report concerning objects wrapped using this notebook.

In [19]:
import fp17
fp17.report(asg)

Headers: 3 (290 SLOC)
Fields: 1 (100.0%)
Methods: 15 (66.67%)
Classes: 4 (100.0%)
